In [4]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [5]:
!python -V

Python 3.10.12


In [6]:
!ls

sample_data


In [7]:
import pickle
import pandas as pd

In [10]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    return df

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')

In [12]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet')

In [13]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [14]:
# Question 1 -5
standard_dev=y_pred.std()
standard_dev

6.353996941249663

In [34]:
year=2023
month=3

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df["result"]=y_pred

df_result= df[['ride_id',"result" ]]

In [36]:
df_result.to_parquet(
    "output_file",
    engine='pyarrow',
    compression=None,
    index=False
)

In [40]:
! ls -l


total 67068
drwx------ 5 root root     4096 Jun 24 07:18  drive
-rw-r--r-- 1 root root    17376 Jun 24 07:21  model.bin
-rw-r--r-- 1 root root 68641704 Jun 24 08:14  output_file
drwxr-xr-x 1 root root     4096 Jun 20 18:46  sample_data
drwxr-xr-x 2 root root     4096 Jun 24 07:16 'Untitled Folder'
